In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem.rdmolfiles import SDWriter
import os

from rdkit.Chem.Lipinski import RotatableBondSmarts

from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmm.app import PDBFile
from openmm import XmlSerializer

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.drawOptions.addStereoAnnotation = False
IPythonConsole.molSize = 500,500

# Load the OpenFF "Sage" force field.
forcefield = ForceField("openff-2.0.0.offxml")
tyk2_list = ['lig_ejm_31', 
            'lig_ejm_42', 
            'lig_ejm_43',
            'lig_ejm_44',
            'lig_ejm_45',
            'lig_ejm_46',
            'lig_ejm_47',
            'lig_ejm_48',
            'lig_ejm_49',
            'lig_ejm_50',
            'lig_ejm_54',
            'lig_ejm_55',
            'lig_jmc_23',
            'lig_jmc_27',
            'lig_jmc_28',
            'lig_jmc_30']

In [ ]:
# retriebe sdf files and save them in ../data/ligands
# convert sdf files to pdf files
# generate serialized system objects for each ligand


# retriev sdf files
import urllib.request
base = 'https://github.com/openforcefield/protein-ligand-benchmark/raw//0.2.1/data/2020-02-07_tyk2/02_ligands/'
for ligand in tyk2_list:
    with urllib.request.urlopen(f"{base}/{ligand}/crd/{ligand}.sdf") as f:
        html = f.read().decode('utf-8')
        with open(f"../data/ligands/{ligand}.sdf", 'w') as f:
            f.write(html)

# load molecules with openff toolkit
molecules = {}
for ligand in tyk2_list:
    print(f'Processing {ligand}')
    m = Molecule.from_file(f"../data/ligands/{ligand}.sdf")
    off_topology = m.to_topology()
    # Parametrize the topology and create an OpenMM System.
    system = forcefield.create_openmm_system(off_topology)
    
    PDBFile.writeFile(off_topology.to_openmm(), m.conformers[0].magnitude/10, open(f"../data/ligands/{ligand}.pdb", 'w'))
    with open(f'../data/ligands/{ligand}_system.xml', 'w') as output:
        output.write(XmlSerializer.serialize(system))


In [ ]:
def flatten_tuples(t):
    for x in t:
        if isinstance(x, tuple):
            yield from flatten_tuples(x)
        else:
            yield x
            

def get_torsion_indices(mol):
    """a helper function to return a sequence of 4-tuple torsions"""
    from rdkit.Chem.Lipinski import RotatableBondSmarts
    
    rot_atom_pairs = mol.GetSubstructMatches(RotatableBondSmarts) # get rotatable bond tuples
    all_atoms = set(list(flatten_tuples(rot_atom_pairs)))
    rot_quadruples = []
    
    for pair in rot_atom_pairs: # query each one
        a1_idx, a2_idx = pair
        a1 = [atom for atom in mol.GetAtoms() if atom.GetIdx() == a1_idx][0]
        a2 = [atom for atom in mol.GetAtoms() if atom.GetIdx() == a2_idx][0]
        a1_nbrs = [nbr for nbr in a1.GetNeighbors()]
        a2_nbrs = [nbr for nbr in a2.GetNeighbors()]
        a1_nbr_indices = [q.GetIdx() for q in a1_nbrs if q.GetAtomicNum() != 1 and q.GetIdx() != a2_idx] # because fuck hydrogens
        a2_nbr_indices = [q.GetIdx() for q in a2_nbrs if q.GetAtomicNum() != 1 and q.GetIdx() != a1_idx]
        
        if len(a1_nbr_indices) == 0 or len(a2_nbr_indices) == 0:
            continue # omit this pair
        
        # iterate over all neighbor combinations and get the quadruple.
        p1, p2, p3, p4 = None, a1_idx, a2_idx, None
        
        # get p1
        for a1_nbr_idx in a1_nbr_indices[::-1]:
            if a1_nbr_idx in all_atoms:
                p1 = a1_nbr_idx
            else:
                continue
        if p1 is None:
            p1 = a1_nbr_idx
        
        # get p4
        for a2_nbr_idx in a2_nbr_indices[::-1]:
            if a2_nbr_idx in all_atoms:
                p4 = a2_nbr_idx
            else:
                continue
        
        if p4 is None:
            p4 = a2_nbr_idx
        
        rot_quadruples.append([p1, p2, p3, p4])
    
    return rot_quadruples


In [ ]:
# collect all the mols
mols = {}
for ligand in tyk2_list:
    print(f'Processing {ligand}')
    suppl = Chem.SDMolSupplier(f"../data/ligands/{ligand}.sdf",  removeHs=False)
    mol = suppl[0]
    mols[ligand] = mol

In [ ]:
all_torsions = {}
for ligand in tyk2_list:
    mol = mols[ligand] # get the mol
    my_torsions = get_torsion_indices(mol)
    all_torsions[ligand] = my_torsions

for ligand in tyk2_list:
    base = f'../data/torsion-scans/{ligand}'
    os.makedirs(base, exist_ok=True)
    for torsion in all_torsions[ligand]:
        torsion_str = '[' + '_'.join([str(q) for q in torsion]) + ']'
        os.makedirs(f'{base}/1D_scan_{torsion_str}', exist_ok=True)
    res = list(zip(all_torsions[ligand], all_torsions[ligand][1:]))
    for torsion1, torsion2 in res:
        torsion1_str = '[' + '_'.join([str(q) for q in torsion1]) + ']'
        torsion2_str = '[' + '_'.join([str(q) for q in torsion2]) + ']'
        os.makedirs(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}', exist_ok=True)


In [ ]:
def generate_psi4_file(m):
    s = 'molecule {\n'
    s += m.to_qcschema().to_string(dtype='psi4', units='angstrom')
    s += '}\n\n'
    s += 'set basis 6-31g*\n'
    s += "gradient('b3lyp')\n"
    return s

In [ ]:
# load molecules with openff toolkit
molecules = {}

for ligand in tyk2_list:
    print(f'Processing {ligand}')
    base = f'../data/torsion-scans/{ligand}'
    m = Molecule.from_file(f"../data/ligands/{ligand}.sdf")
    off_topology = m.to_topology()
    # Parametrize the topology and create an OpenMM System.
    system = forcefield.create_openmm_system(off_topology)

    for torsion in all_torsions[ligand]:
        torsion_str = '[' + '_'.join([str(q) for q in torsion]) + ']'
        PDBFile.writeFile(off_topology.to_openmm(), m.conformers[0].magnitude, open(f"{base}/1D_scan_{torsion_str}/{ligand}.pdb", 'w'))
        # generate the xml file
        with open(f'{base}/1D_scan_{torsion_str}/{ligand}_system.xml', 'w') as output:
            output.write(XmlSerializer.serialize(system))
        # generate the constraints file
        with open(f'{base}/1D_scan_{torsion_str}/constraints.txt', 'w') as output:
            output.write('$scan\n')
            output.write(f'dihedral {torsion[0]} {torsion[1]} {torsion[2]} {torsion[3]} -180 180 60\n')
        # generate the run files for openMM/ANI
        with open(f'{base}/1D_scan_{torsion_str}/run_openmm.sh', 'w') as output:
            output.write(f'geometric-optimize --engine openmm --pdb {ligand}.pdb {ligand}_system.xml constraints.txt')
        # generate the run files for psi4
        with open(f'{base}/1D_scan_{torsion_str}/run_psi4.sh', 'w') as output:
            output.write(f'geometric-optimize run.psi4in constraints.txt --engine psi4 --nt 4 --coordsys dlc')
        # generate the psi4 input file
        s = generate_psi4_file(m)
        with open(f'{base}/1D_scan_{torsion_str}/run.psi4in', 'w') as output:
            output.write(s)

    res = list(zip(all_torsions[ligand], all_torsions[ligand][1:]))
    for torsion1, torsion2 in res:
        torsion1_str = '[' + '_'.join([str(q) for q in torsion1]) + ']'
        torsion2_str = '[' + '_'.join([str(q) for q in torsion2]) + ']'
        PDBFile.writeFile(off_topology.to_openmm(), m.conformers[0].magnitude, open(f"{base}/2D_scan_{torsion1_str}_{torsion2_str}/{ligand}.pdb", 'w'))
        # generate the xml file
        with open(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}/{ligand}_system.xml', 'w') as output:
            output.write(XmlSerializer.serialize(system))
        # generate the constraints file
        with open(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}/constraints.txt', 'w') as output:
            output.write('$scan\n')
            output.write(f'dihedral {torsion1[0]} {torsion1[1]} {torsion1[2]} {torsion1[3]} -180 180 60\n')
            output.write(f'dihedral {torsion2[0]} {torsion2[1]} {torsion2[2]} {torsion2[3]} -180 180 60\n')
        # generate the run files for openMM/ANI
        with open(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}/run_openmm.sh', 'w') as output:
            output.write(f'geometric-optimize --engine openmm --pdb {ligand}.pdb {ligand}_system.xml constraints.txt')
        # generate the run files for psi4
        with open(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}/run_psi4.sh', 'w') as output:
            output.write(f'geometric-optimize run.psi4in constraints.txt --engine psi4 --nt 4 --coordsys dlc')
        # generate the psi4 input file
        s = generate_psi4_file(m)
        with open(f'{base}/2D_scan_{torsion1_str}_{torsion2_str}/run.psi4in', 'w') as output:
            output.write(s)
